# EEG preprocessing with MNE

Imports

In [1]:
import mne
import numpy as np
import pandas as pd
from autoreject import AutoReject
ar = AutoReject()
import copy

In [2]:
#create dataframe from participants.tsv containing columns 1 and 2
df_participants = pd.read_csv('../ds003944/participants.tsv', sep='\t', usecols=[0,1])
print(df_participants)

   participant_id       type
0        sub-1448    Control
1        sub-1824  Psychosis
2        sub-1971    Control
3        sub-1983  Psychosis
4        sub-1989    Control
..            ...        ...
77      sub-2184A  Psychosis
78      sub-2193A    Control
79      sub-2214A    Control
80      sub-2217A  Psychosis
81      sub-2221A    Control

[82 rows x 2 columns]


In [3]:
#create a dictionary with new channel names
channels_new_names_dict = {
    'EEG019':'FC5',
    'EEG020':'FC1',
    'EEG021':'FC2',
    'EEG022':'FC6',
    'EEG027':'C5',
    'EEG028':'C3',
    'EEG029':'C1',
    'EEG030':'Cz',
    'EEG031':'C2',
    'EEG032':'C4',
    'EEG033':'C6',
    'EEG038':'CP3',
    'EEG039':'CP1',
    'EEG040': 'CP2',
    'EEG041': 'CP4',
    'EEG001': 'Fp1',
    'EEG002': 'Fpz',
    'EEG003': 'Fp2',
    'EEG004': 'AF7',
    'EEG005': 'AF3',
    'EEG006': 'AF4',
    'EEG008': 'F7',
    'EEG009': 'F5',
    'EEG010': 'F3',
    'EEG011': 'F1',
    'EEG012': 'Fz',
    'EEG013': 'F2',
    'EEG014': 'F4',
    'EEG015': 'F6',
    'EEG016': 'F8',
    'EEG018': 'FT7',
    'EEG023': 'FT8',
    'EEG034': 'T8',
    'EEG035': 'T10',
    'EEG045': 'P5',
    'EEG046': 'P3',
    'EEG047': 'P1',
    'EEG048': 'Pz',
    'EEG049': 'P2',
    'EEG050': 'P4',
    'EEG051': 'P6',
    'EEG052': 'P8',
    'EEG053': 'PO7',
    'EEG054': 'PO3',
    'EEG055': 'PO4',
    'EEG056': 'PO8',
    'EEG057': 'O1',
    'EEG058': 'Oz',
    'EEG059': 'O2',
    'EEG060': 'Iz',
    "EEG007": 'AF6',
    "EEG017": 'FT9',
    "EEG024": 'FT10',
    "EEG025": 'T9',
    "EEG026": 'T7',
    "EEG042": 'TP8',
    "EEG043": 'TP10',
    "EEG044": 'P7',
    "EEG061": 'VEOG',
    "EEG062": 'Misc',
    "EEG063": 'ECG',
    "EEG064": 'M2',
    "EEG036": 'TP9',
    "EEG037": 'TP7'
}

In [4]:
df_participants = df_participants.drop(index=range(69))

In [5]:
print(df_participants)

   participant_id       type
69       sub-2221    Control
70       sub-2222    Control
71       sub-2223    Control
72      sub-2140A    Control
73      sub-2170A  Psychosis
74      sub-2174A  Psychosis
75      sub-2176A  Psychosis
76      sub-2177A  Psychosis
77      sub-2184A  Psychosis
78      sub-2193A    Control
79      sub-2214A    Control
80      sub-2217A  Psychosis
81      sub-2221A    Control


In [6]:
#run the below code for each subject using vmrk files within folder that name equals value in column 1 of df_participants;
#if loops is interrupted, it can be continued from the next subject



for index, row in df_participants.iterrows():
    
    try:

        #reading brainvision files
        brainvision_raw = mne.io.read_raw_brainvision(
            f'../ds003944/{row["participant_id"]}/eeg/{row["participant_id"]}_task-Rest_eeg.vhdr',
            preload=True)

        brainvision_fname = f'../ds003944/{row["participant_id"]}/eeg/{row["participant_id"]}_task-Rest_eeg.vhdr'

        
        #renaming channels
        brainvision_raw.rename_channels(channels_new_names_dict)

        #setting channel types
        brainvision_raw.set_channel_types({
            'VEOG':'eog',
            'ECG':'ecg',
            'Misc':'misc',
            'M2':'eeg'
        })

        #setting montage
        brainvision_raw.set_montage('standard_1020')

        #filtering data
        mat_nyquist_freq = brainvision_raw.info["sfreq"] / 2
        brainvision_raw.notch_filter(freqs=np.arange(50, mat_nyquist_freq, 50), picks='eeg')
        brainvision_raw.filter(0.05, 30, method="iir", picks='eeg')

        #setting reference
        reference = 'average'
        brainvision_raw.set_eeg_reference(ref_channels=reference)
        reference = ['M2']
        brainvision_raw.set_eeg_reference(ref_channels=reference)

        #data segmentation into epochs
        events_from_annotations, events_dict_from_annotations = mne.events_from_annotations(brainvision_raw)
        print(events_dict_from_annotations)
        mat_epochs = mne.make_fixed_length_epochs(brainvision_raw, duration=30, preload=True)

        #occular correction with ICA
        ica = mne.preprocessing.ICA(n_components=50, random_state=42, method="infomax")
        ica.fit(mat_epochs)
        ica.exclude=[]
        eog_indices, eog_sources = ica.find_bads_eog(
            mat_epochs, 
            measure='zscore', 
            threshold='auto'
        )
        ica.exclude = eog_indices
        ica.apply(mat_epochs)
    
        #interpolating bad channels
        mat_epochs.info['bads'] = ['Fp1', 'Fp2', 'AF3', 'AF4']
        mat_epochs = mat_epochs.interpolate_bads(reset_bads=False)

        #artifact rejection
        mat_epochs_clean = ar.fit_transform(mat_epochs)

        #saving epochs to fif file
        dir_to_store_fif = "../ds003944/preprocessed-data-autoreject/"
        start = brainvision_fname.index("sub-")
        end = brainvision_fname.index("/eeg")
        sub_id = brainvision_fname[start:end]
        mat_epochs_clean.save(f"{dir_to_store_fif}{sub_id}-epo.fif")

        #update df_participants with number of epochs and fif file name
        df_participants.loc[index, "number_of_epochs"] = len(mat_epochs)
        df_participants.loc[index, "fif_file_name"] = f"{sub_id}-epo.fif"
        

    except:
        continue

    #saving df_participants to csv file
    df_participants.to_csv("../ds003944/preprocessed-data-autoreject/df_participants.csv", index=False)

        





    



Extracting parameters from ../ds003944/sub-2221/eeg/sub-2221_task-Rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 303999  =      0.000 ...   303.999 secs...
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 6601 samples (6.601 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.05 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.05, 30.00 Hz: -6.02, -6.02 dB



[Parallel(n_jobs=1)]: Done  61 out of  61 | elapsed:    1.0s finished


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
{}
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 10 events and 30000 original time points ...
0 bad epochs dropped
Fitting ICA to data using 61 channels (please be patient, this may take a while)
Selecting by number: 50 components
 
Fitting ICA took 437.6s.
Using EOG channel: VEOG
Applying ICA to Epochs instance
    Transforming to ICA space (50 components)
    Zeroing out 3 ICA components
    Projecting back using 61 PCA components
Interpolating bad channels
    Automatic origin fit: head of radius 97.1 mm
Computing interpolation matrix from 57 sensor positions
Interpolating 4 sensors
Running autoreject on ch_type=eeg


...

  0%|          | Fold : 0/10 [00:00<?,       ?it/s]





Estimated consensus=0.60 and n_interpolate=32


c:\dev\python\.venv\Lib\site-packages\autoreject\utils.py:65: UserWarning: 4 channels are marked as bad. These will be ignored. If you want them to be considered by autoreject please remove them from epochs.info["bads"].
  warnings.warn(


  0%|          | Repairing epochs : 0/10 [00:00<?,       ?it/s]

Dropped 1 epoch: 9
Extracting parameters from ../ds003944/sub-2221A/eeg/sub-2221A_task-Rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 317999  =      0.000 ...   317.999 secs...
